https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks/

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
# Import MNST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/', one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [2]:
# parameters
learning_rate = 0.001
training_iters = 200000 # max training iteration
batch_size = 128
display_step = 10

In [3]:
# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
p_dropout = 0.75 # Dropout, probability to keep units

In [4]:
# graph input
x = tf.placeholder('float',[None, n_input])
y = tf.placeholder('float',[None, n_classes])
keep_prob = tf.placeholder('float')

In [5]:
# variables to use
weights = {
    # 5*5 conv layer, 1 input, 32 output (28+5-1=32)
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    # 5*5 conv layer, 32 inputs, 64 output (28+5-1=32)
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),
    # fully connected layer, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # output layer, 1024 inputs, 10 outputs
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [6]:
# define the model, wrapper1 -- conv2d
def conv2d(x, W, b, strides=1):
    '''
    Here is an explination of the strides parameter in conv2d: 
    http://stackoverflow.com/questions/34642595/tensorflow-strides-argument
    >the input tensor has 4 dimensions:  [batch, height, width, channels]
    
    tf.nn.bias_add is almost the same as tf.add
    https://www.tensorflow.org/versions/r0.10/api_docs/python/nn.html#bias_add
    '''
    out = tf.nn.conv2d(input=x, filter=W, strides=[1, strides, strides, 1], padding='SAME')
    out = tf.nn.bias_add(out, b)
    return tf.nn.relu(out)

In [7]:
# define the model, wrapper2 -- max pooling
def maxpool2d(x, k=2):
    return tf.nn.max_pool(value=x, ksize=[1,k,k,1], strides=[1,k,k,1],padding='SAME')

In [8]:
# define the model
def convnet(x, weights, biases, dropout):
    x = tf.reshape(x, [-1, 28, 28, 1])
    # conv layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1)
    # conv layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2)
    # fully connected layer
    fc1 = tf.reshape( conv2, 
                    shape=(-1, weights['wd1'].get_shape().as_list()[0])  )
    fc1 = tf.matmul(fc1, weights['wd1']) + biases['bd1']
    fc1 = tf.nn.relu(fc1)
    # dropout
    fc1 = tf.nn.dropout(fc1, keep_prob=dropout)
    # output layer: output logits
    out = tf.matmul(fc1, weights['out']) + biases['out']
    return out

pred = convnet(x, weights, biases, keep_prob)

In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [10]:
correct_pred = tf.equal( tf.argmax(pred,1), tf.argmax(y,1) )
accuracy = tf.reduce_mean(tf.cast(correct_pred, 'float'))

In [11]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    step = 1
    while step*batch_size<training_iters:
        xs, ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x:xs, y:ys, keep_prob:p_dropout})
        if step % display_step == 0: 
            loss, acc = sess.run([cost, accuracy], feed_dict={x:xs, y:ys, keep_prob:1.0}) # why 1.0 ? 
            print 'iter %d, minibatch loss=%.6f, accuracy=%.5f' % (step*batch_size, loss, acc)
        step += 1
    print 'finish!'
    print 'test set accuracy=%.9f' % sess.run(accuracy, 
                                              feed_dict={x: mnist.test.images[:256],
                                                        y: mnist.test.labels[:256],
                                                        keep_prob: 1.0})

iter 1280, minibatch loss=21012.941406, accuracy=0.25781
iter 2560, minibatch loss=12544.046875, accuracy=0.49219
iter 3840, minibatch loss=8028.247559, accuracy=0.60156
iter 5120, minibatch loss=4436.107910, accuracy=0.78906
iter 6400, minibatch loss=3245.677979, accuracy=0.79688
iter 7680, minibatch loss=7462.788574, accuracy=0.71875
iter 8960, minibatch loss=3210.405029, accuracy=0.83594
iter 10240, minibatch loss=2923.085938, accuracy=0.81250
iter 11520, minibatch loss=1718.688721, accuracy=0.85938
iter 12800, minibatch loss=3295.904053, accuracy=0.80469
iter 14080, minibatch loss=1269.590454, accuracy=0.92969
iter 15360, minibatch loss=1638.359985, accuracy=0.92969
iter 16640, minibatch loss=1812.238525, accuracy=0.87500
iter 17920, minibatch loss=1454.659912, accuracy=0.90625
iter 19200, minibatch loss=926.651306, accuracy=0.91406
iter 20480, minibatch loss=577.515686, accuracy=0.92188
iter 21760, minibatch loss=2996.989014, accuracy=0.85156
iter 23040, minibatch loss=1050.532837